In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
# Carregar o conjunto de dados Iris
iris = load_iris()
X = iris.data
y = iris.target

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Padronizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Data Augmentation: Ruído gaussiano
================
Conjunto de técnicas que evitam Overfitting (hiper-ajuste aos dados de treinamento, ou seja, aprende-se os dados treinados, não as características do problema).

O que é Ruído Gaussiano?
_______________

* Gerado a partir de uma distribuição normal, caracterizada por uma média $\mu$ e um desvio-padrão $\sigma$;
* O ruído deve ser pequeno o suficiente para não distorcer significativamente os dados, mas grandeo suficiente para criar variações úteis

Adicão de ruído gaussiano
___________________________

* Para cada ponto de dados original, crie uma nova amostra adicionando um ruído gaussiano. A nova amostra será:

$$ X_novo = X_original + \mathcal{E}$$

Onde o ruído gaussiano $\mathcal{E}$:

* $\mathcal{E} \sim \mathcal{N}(0, \sigma^2)$
* $\sigma$ deve ser escolhido com base na escala e na sensibilidade dos dados

Escolha do valor adequado para $\sigma$
__________________________________________

* Se o desvio padrão do ruído for muito pequeno, as novas amostras serão muito semelhantes aos dados originais, oferecendo pouca diversidade;
* Se for muito grande, o ruído pode introduzir uma variabilidade que não está presente nos dados reais, potencialmente corrompendo a integridade dos dados;
* Em geral, um bom ponto de partida para o desvio padrão do ruído é cerca de 1-5% da faixa (range) de cada feature.

In [5]:
# Função para adicionar ruído gaussiano
def add_gaussian_noise(data, sigma=0.05):
    noise = np.random.normal(0, sigma, data.shape)
    return data + noise

# Adicionar ruído aos dados d2. ______________________e treino
X_train_noisy = add_gaussian_noise(X_train, sigma=0.05)

# Combinar os dados originais com os dados aumentados
X_train_augmented = np.vstack((X_train, X_train_noisy))
y_train_augmented = np.hstack((y_train, y_train))  # Duplicar os rótulos para corresponder ao tamanho aumentado

# Converter para tensores do PyTorch
X_train_tensor_augmented = torch.tensor(X_train_augmented, dtype=torch.float32)
y_train_tensor_augmented = torch.tensor(y_train_augmented, dtype=torch.long)

# Criar um DataLoader com os dados aumentados
train_dataset_augmented = TensorDataset(X_train_tensor_augmented, y_train_tensor_augmented)
train_loader_augmented = DataLoader(dataset=train_dataset_augmented, batch_size=32, shuffle=True)


In [6]:
class IrisClassificationModel(nn.Module):
    def __init__(self):
        super(IrisClassificationModel, self).__init__()
        self.hidden = nn.Linear(X_train.shape[1], 16)  # Camada oculta com 16 neurônios
        self.output = nn.Linear(16, 3)  # Três classes de saída

    def forward(self, x):
        x = torch.relu(self.hidden(x))  # Função de ativação ReLU na camada oculta
        x = self.output(x)  # Saída direta (softmax será aplicado na função de perda)
        return x

model = IrisClassificationModel()

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [8]:
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, targets in train_loader_augmented:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass e otimização
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Mostrar o progresso a cada 10 épocas
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 0.8935
Epoch [20/100], Loss: 0.6861
Epoch [30/100], Loss: 0.5534
Epoch [40/100], Loss: 0.4953
Epoch [50/100], Loss: 0.4801
Epoch [60/100], Loss: 0.3202
Epoch [70/100], Loss: 0.4323
Epoch [80/100], Loss: 0.3968
Epoch [90/100], Loss: 0.3031
Epoch [100/100], Loss: 0.3087


In [9]:
# Converter os dados de teste para tensores do PyTorch
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Avaliar o modelo no conjunto de teste
model.eval()  # Colocar o modelo em modo de avaliação
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)
    print(f'Accuracy on the test set: {accuracy * 100:.2f}%')


Accuracy on the test set: 93.33%


Data Augmentation: Mixup
================================

técnica que cria novas amostras de treinamento ao combinar pares de amostras existentes, tanto em termos de features quanto de rótulos. Essa técnica é particularmente eficaz em melhorar a robustez e a generalização do modelo, especialmente em tarefas de classificação.

Fórmulas do Mixup
_____________________
A ideia central do Mixup é criar novas amostras lineares interpolando entre dois exemplos aleatórios do conjunto de dados.
* Para duas amostras $(x_i , y_i)$ e $(x_j , y_j)$, uma nova amostra $(\tilde{x}, \tilde{y})$ é criada da seguinte forma:

$$\tilde{x} = \lambda x_i + (1 - \lambda) x_j$$
$$\tilde{y} = \lambda y_i + (1 - \lambda) y_j$$

Onde $\lambda$ é um valor retirado de uma distribuição beta $\beta (\alpha , \alpha)$, com $\alpha$ sendo um hiperparâmetro que contra a mistura

In [10]:
# Converter rótulos para one-hot encoding
y_train_onehot = np.eye(3)[y_train]  # 3 classes no Iris
y_test_onehot = np.eye(3)[y_test]

In [11]:
def mixup_data(x, y, alpha=1.0):
    """Retorna as novas amostras e rótulos após aplicação do Mixup"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.shape[0]
    index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    mixed_y = lam * y + (1 - lam) * y[index, :]

    return mixed_x, mixed_y

# Converter para tensores do PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_onehot, dtype=torch.float32)

# Aplicar Mixup durante o treinamento
class MixupDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, alpha=1.0):
        self.X = X
        self.y = y
        self.alpha = alpha

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x, y = self.X[idx], self.y[idx]
        return mixup_data(x.unsqueeze(0), y.unsqueeze(0), self.alpha)

train_dataset_mixup = MixupDataset(X_train_tensor, y_train_tensor, alpha=0.4)
train_loader_mixup = DataLoader(dataset=train_dataset_mixup, batch_size=32, shuffle=True)


In [12]:
num_epochs = 100
for epoch in range(num_epochs):
    for (inputs, targets) in train_loader_mixup:
        # Extrair mixup
        inputs = inputs.squeeze(0)
        targets = targets.squeeze(0)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass e otimização
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Mostrar o progresso a cada 10 épocas
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: -0.0000
Epoch [20/100], Loss: -0.0000
Epoch [30/100], Loss: -0.0000
Epoch [40/100], Loss: -0.0000
Epoch [50/100], Loss: -0.0000
Epoch [60/100], Loss: -0.0000
Epoch [70/100], Loss: -0.0000
Epoch [80/100], Loss: -0.0000
Epoch [90/100], Loss: -0.0000
Epoch [100/100], Loss: -0.0000


In [13]:
# Converter os dados de teste para tensores do PyTorch
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_onehot, dtype=torch.float32)

# Avaliar o modelo no conjunto de teste
model.eval()  # Colocar o modelo em modo de avaliação
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == torch.argmax(y_test_tensor, dim=1)).sum().item() / y_test_tensor.size(0)
    print(f'Accuracy on the test set: {accuracy * 100:.2f}%')


Accuracy on the test set: 93.33%


Data Augmentation: SMOTE (Synthetic Minority Over-sampling Technique)
====================================================================

Técnica de aumento de dados que gera novos exemplos sintéticos para a classe minoritária, com o objetivo de equilibrar o conjunto de dados.

In [14]:
from imblearn.over_sampling import SMOTE

In [15]:
# Aplicar SMOTE para gerar novos exemplos sintéticos
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [16]:
# Converter os dados aumentados para tensores do PyTorch
X_train_tensor_smote = torch.tensor(X_train_smote, dtype=torch.float32)
y_train_tensor_smote = torch.tensor(y_train_smote, dtype=torch.long)

In [17]:
from torch.utils.data import DataLoader, TensorDataset

# Criar um DataLoader com os dados aumentados
train_dataset_smote = TensorDataset(X_train_tensor_smote, y_train_tensor_smote)
train_loader_smote = DataLoader(dataset=train_dataset_smote, batch_size=32, shuffle=True)

In [18]:
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, targets in train_loader_smote:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass e otimização
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Mostrar o progresso a cada 10 épocas
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 0.3736
Epoch [20/100], Loss: 0.3213
Epoch [30/100], Loss: 0.2442
Epoch [40/100], Loss: 0.2607
Epoch [50/100], Loss: 0.2748
Epoch [60/100], Loss: 0.2816
Epoch [70/100], Loss: 0.2175
Epoch [80/100], Loss: 0.2530
Epoch [90/100], Loss: 0.2723
Epoch [100/100], Loss: 0.1755


In [19]:
# Converter os dados de teste para tensores do PyTorch
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Avaliar o modelo no conjunto de teste
model.eval()  # Colocar o modelo em modo de avaliação
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)
    print(f'Accuracy on the test set: {accuracy * 100:.2f}%')


Accuracy on the test set: 100.00%
